# Naive approach random_forest + optuna - holidays

In [9]:
model_name = 'default_xgboost_with_holidays_seasonality_features'

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data_path = "./data/"
df_train = pd.read_csv(data_path + "train.csv")
df_test = pd.read_csv(data_path + "test.csv")

# drop id
df_train.drop('id', axis=1, inplace=True)

df_train.head(10)

,date,country,store,product,num_sold
0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49
5,2017-01-01,Argentina,Kaggle Store,Using LLMs to Improve Your Coding,88
6,2017-01-01,Argentina,Kaggle Store,Using LLMs to Train More LLMs,98
7,2017-01-01,Argentina,Kaggle Store,Using LLMs to Win Friends and Influence People,14
8,2017-01-01,Argentina,Kaggle Store,Using LLMs to Win More Kaggle Competitions,83
9,2017-01-01,Argentina,Kaggle Store,Using LLMs to Write Better,69


### Features

In [11]:
import holidays

def transform_date(df):
    df['date'] = pd.to_datetime(df['date'])
    return df

def create_features(df):
    # split date into year, month, day
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day

    # week number
    df['week_number'] = df['date'].dt.isocalendar().week.astype(int)
    
    # day of week
    df['dayofweek'] = df['date'].dt.dayofweek
    df['dayofmonth'] = df['date'].dt.day
    # weekend
    #df['weekend'] = (df['date'].dt.weekday >=4).astype(int)

    # drop date
    #df = df.drop('date', axis=1)

    # one-hot encoding of 'country', 'store', 'product' columns
    #df_train = pd.get_dummies(df_train, columns=['country', 'store', 'product'])


    df['country'] = df['country'].astype('category')
    df['store'] = df['store'].astype('category')
    df['product'] = df['product'].astype('category')

    return df

def encode_cat_variables(df):
    categorical_features = ["country", "store", "product"]
    df = pd.get_dummies(df, columns=categorical_features)
    return df

def add_holidays(df):
    # courtesy of kacperrabczewski in https://www.kaggle.com/code/kacperrabczewski/last-minute-forecasting#Modeling-%F0%9F%AA%84

    years_list = [2017, 2018, 2019, 2020, 2021, 2022, 2023]

    Argentina_holidays = holidays.CountryHoliday('AR', years=years_list)
    Canada_holidays = holidays.CountryHoliday('CA', years=years_list)
    Estonia_holidays = holidays.CountryHoliday('EE', years=years_list)
    Japan_holidays = holidays.CountryHoliday('JP', years=years_list)
    Spain_holidays = holidays.CountryHoliday('ES', years=years_list)
    
    # Create Holiday Column 
    df['Argentina_holiday'] = df.loc[df['country'] == 'Argentina', 'date'].apply(lambda d: True if d in Argentina_holidays else False)
    df['Canada_holiday'] = df.loc[df['country'] == 'Canada', 'date'].apply(lambda d: True if d in Canada_holidays else False)
    df['Estonia_holiday'] = df.loc[df['country'] == 'Estonia', 'date'].apply(lambda d: True if d in Estonia_holidays else False)
    df['Japan_holiday'] = df.loc[df['country'] == 'Japan', 'date'].apply(lambda d: True if d in Japan_holidays else False)
    df['Spain_holiday'] = df.loc[df['country'] == 'Spain', 'date'].apply(lambda d: True if d in Spain_holidays else False)

    df['Argentina_holiday'] = df['Argentina_holiday'].fillna(False)
    df['Canada_holiday'] = df['Canada_holiday'].fillna(False)
    df['Estonia_holiday'] = df['Estonia_holiday'].fillna(False)
    df['Japan_holiday'] = df['Japan_holiday'].fillna(False)
    df['Spain_holiday'] = df['Spain_holiday'].fillna(False)

    df = df.drop('date', axis=1)

    return df

def seasonality_features(df):
    
    # getting seasonal patterns
    df['month_sin'] = np.sin(2*np.pi*df.month/12)
    df['month_cos'] = np.cos(2*np.pi*df.month/12)
    df['day_sin'] = np.sin(2*np.pi*df.dayofmonth/31)
    df['day_cos'] = np.cos(2*np.pi*df.dayofmonth/31)
    return df

df_train_0 = transform_date(df_train)
df_train_0 = create_features(df_train_0)
df_train_0 = add_holidays(df_train_0)
df_train_0 = seasonality_features(df_train_0)
df_train_0 = encode_cat_variables(df_train_0)
df_train_0.head()

,num_sold,year,month,day,week_number,dayofweek,dayofmonth,Argentina_holiday,Canada_holiday,Estonia_holiday,...,country_Japan,country_Spain,store_Kagglazon,store_Kaggle Learn,store_Kaggle Store,product_Using LLMs to Improve Your Coding,product_Using LLMs to Train More LLMs,product_Using LLMs to Win Friends and Influence People,product_Using LLMs to Win More Kaggle Competitions,product_Using LLMs to Write Better
0,63,2017,1,1,52,6,1,True,False,False,...,False,False,False,True,False,True,False,False,False,False
1,66,2017,1,1,52,6,1,True,False,False,...,False,False,False,True,False,False,True,False,False,False
2,9,2017,1,1,52,6,1,True,False,False,...,False,False,False,True,False,False,False,True,False,False
3,59,2017,1,1,52,6,1,True,False,False,...,False,False,False,True,False,False,False,False,True,False
4,49,2017,1,1,52,6,1,True,False,False,...,False,False,False,True,False,False,False,False,False,True


In [12]:
model_features = df_train_0.columns.tolist()
model_features.remove("num_sold")
model_features

['year',
 'month',
 'day',
 'week_number',
 'dayofweek',
 'dayofmonth',
 'Argentina_holiday',
 'Canada_holiday',
 'Estonia_holiday',
 'Japan_holiday',
 'Spain_holiday',
 'month_sin',
 'month_cos',
 'day_sin',
 'day_cos',
 'country_Argentina',
 'country_Canada',
 'country_Estonia',
 'country_Japan',
 'country_Spain',
 'store_Kagglazon',
 'store_Kaggle Learn',
 'store_Kaggle Store',
 'product_Using LLMs to Improve Your Coding',
 'product_Using LLMs to Train More LLMs',
 'product_Using LLMs to Win Friends and Influence People',
 'product_Using LLMs to Win More Kaggle Competitions',
 'product_Using LLMs to Write Better']

In [13]:
df_train_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 29 columns):
 #   Column                                                  Non-Null Count   Dtype  
---  ------                                                  --------------   -----  
 0   num_sold                                                136950 non-null  int64  
 1   year                                                    136950 non-null  int32  
 2   month                                                   136950 non-null  int32  
 3   day                                                     136950 non-null  int32  
 4   week_number                                             136950 non-null  int32  
 5   dayofweek                                               136950 non-null  int32  
 6   dayofmonth                                              136950 non-null  int32  
 7   Argentina_holiday                                       136950 non-null  bool   
 8   Canada_holiday          

Function by which submissions are scored is SMAPE:

In [14]:
def SMAPE(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [20]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb
import optuna

train_index, val_index = [ x for x in TimeSeriesSplit(n_splits=5).split(df_train_0) ][-1]

X_train = df_train_0.iloc[train_index][model_features]
y_train = df_train_0.iloc[train_index]['num_sold']

X_val = df_train_0.iloc[val_index][model_features]
y_val = df_train_0.iloc[val_index]['num_sold']

cat_features_indices = np.where((X_train.dtypes == "category") | (X_train.dtypes == "object"))[0]
cat_features_indices

def objective(trial):
    model= xgb.XGBRegressor(
        tree_method="gpu_hist", enable_categorical=True,
        n_estimators=trial.suggest_int('n_estimators', 100, 1000),
        max_depth=trial.suggest_int('max_depth', 3, 15),
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[( X_train, y_train), ( X_val, y_val)],
        verbose=False
    )

    return SMAPE(y_val, np.round(model.predict(X_val)))


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_hyperparams = study.best_params

[I 2023-07-29 18:21:05,210] A new study created in memory with name: no-name-71e6d43b-cbb5-4f8e-b388-a54f4027cebb


[I 2023-07-29 18:22:26,576] Trial 0 finished with value: 11.049948987550778 and parameters: {'n_estimators': 796, 'max_depth': 15}. Best is trial 0 with value: 11.049948987550778.
[I 2023-07-29 18:23:24,821] Trial 1 finished with value: 10.413084585617982 and parameters: {'n_estimators': 858, 'max_depth': 12}. Best is trial 1 with value: 10.413084585617982.
[I 2023-07-29 18:23:29,529] Trial 2 finished with value: 14.329526419456538 and parameters: {'n_estimators': 893, 'max_depth': 5}. Best is trial 1 with value: 10.413084585617982.
[I 2023-07-29 18:24:48,407] Trial 3 finished with value: 11.180639259363272 and parameters: {'n_estimators': 944, 'max_depth': 13}. Best is trial 1 with value: 10.413084585617982.
[I 2023-07-29 18:26:07,785] Trial 4 finished with value: 11.049948987550778 and parameters: {'n_estimators': 614, 'max_depth': 15}. Best is trial 1 with value: 10.413084585617982.
[I 2023-07-29 18:26:16,580] Trial 5 finished with value: 11.670464870086745 and parameters: {'n_estim

KeyboardInterrupt: 

In [23]:
best_hyperparams

{'n_estimators': 810, 'max_depth': 12}

In [24]:
# split into train and validation

results_smape = []
results_r2 = []
results_mse = []


N_SPLITS = 5

for train_index, val_index in TimeSeriesSplit(n_splits=N_SPLITS).split(df_train_0):
    X_train = df_train_0.iloc[train_index][model_features]
    y_train = df_train_0.iloc[train_index]['num_sold']

    X_val = df_train_0.iloc[val_index]
    y_val = df_train_0.iloc[val_index]['num_sold']

    model = xgb.XGBRegressor(tree_method="gpu_hist", enable_categorical=True)
    model.fit(
        X_train, y_train,
        #eval_set=[( X_train, y_train), ( X_val, y_val)],
        verbose=False
    )

    # evaluate on validation set
    y_pred = np.round(model.predict(X_val[model_features]))

    smape = SMAPE(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    mse = mean_squared_error(y_val, y_pred)

    results_smape.append(smape)
    results_r2.append(r2)
    results_mse.append(mse)
    del model

print("Cross validated results:")

print("SMAPE: %s" % results_smape)
print("R2: %s" % results_r2)
print("MSE: %s" % results_mse)

print("Mean results:")
print("SMAPE: %.4f" % np.mean(results_smape))
print("R2: %.4f" % np.mean(results_r2))
print("MSE: %.4f" % np.mean(results_mse))


# add a row with results to csv file leaderboard.csv

row = [model_name, np.mean(results_smape), np.mean(results_r2), np.mean(results_mse)]
with open('leaderboard.csv', 'a') as fd:
    import csv
    fd.write('\n')
    writer = csv.writer(fd, delimiter=',', lineterminator=';\n')
    writer.writerow(row)

Cross validated results:
SMAPE: [13.85144387499987, 9.271005920021572, 11.780750994602853, 18.984407960786655, 14.560680492771942]
R2: [0.9631319421575548, 0.9871692628195864, 0.9708657124348878, 0.9462368588893403, 0.9746859805294681]
MSE: [1299.9037897042717, 448.10865279299014, 916.5842716319825, 1697.9399342825848, 992.0135377875137]
Mean results:
SMAPE: 13.6897
R2: 0.9684
MSE: 1070.9100


In [25]:
import datetime

# train and predict for submission

df_train = pd.read_csv(data_path + "train.csv")
df_train['date'] = pd.to_datetime(df_train['date'])

df_train_0 = transform_date(df_train)
df_train_0 = create_features(df_train_0)
df_train_0 = add_holidays(df_train_0)
df_train_0 = seasonality_features(df_train_0)
df_train_0 = encode_cat_variables(df_train_0)

X_train = df_train_0.drop('num_sold', axis=1)
y_train = df_train_0['num_sold']


model = xgb.XGBRegressor(tree_method="gpu_hist", enable_categorical=True)
model.fit(
    X_train[model_features], y_train,
    verbose=False
)

df_val = X_val.copy()
df_val['actual'] = y_val
df_val['prediction'] = model.predict(df_val[model_features])
df_val['prediction'] = df_val['prediction'].astype(int)


X_train['date'] = pd.to_datetime(X_train['year'].astype(str) + '-' + X_train['month'].astype(str) + '-' + X_train['day'].astype(str))
X_val['date'] = pd.to_datetime(X_val['year'].astype(str) + '-' + X_val['month'].astype(str) + '-' + X_val['day'].astype(str))
df_val['date'] = pd.to_datetime(df_val['year'].astype(str) + '-' + df_val['month'].astype(str) + '-' + df_val['day'].astype(str))

# sort by date
X_train.sort_values(by='date', inplace=True)
X_val.sort_values(by='date', inplace=True)

# plot 'num_sold' vs 'date'
fig, ax = plt.subplots(figsize=(15, 5))

ax.plot(X_train['date'], y_train, label='train', color='lightblue')
ax.plot(X_val['date'], y_val, label='validation', color='pink')
ax.plot(df_val['date'], df_val['prediction'], label='prediction', color='crimson')

# x axis show only from 2021 onwards
ax.set_xlim([datetime.date(2021, 1, 1), df_val['date'].max()])
ax.set_xlabel('date')
ax.set_ylabel('num_sold')

ax.legend()
plt.show()

In [27]:
# test 

df_test = pd.read_csv(data_path + "test.csv")
df_test_0 = transform_date(df_test)
df_test_0 = create_features(df_test_0)
df_test_0 = add_holidays(df_test_0)
df_test_0 = seasonality_features(df_test_0)
df_test_0 = encode_cat_variables(df_test_0)

# predict for test set
df_test['prediction'] = model.predict(df_test_0[model_features])
df_test['prediction'] = df_test['prediction'].astype(int)
display(df_test)

df_test['num_sold'] = df_test['prediction']
df_test[['id', 'num_sold']].to_csv(f'{model_name}_submission.csv', index=False)

,id,date,country,store,product,year,month,day,week_number,dayofweek,dayofmonth,Argentina_holiday,Canada_holiday,Estonia_holiday,Japan_holiday,Spain_holiday,prediction
0,136950,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,2022,1,1,52,5,1,True,False,False,False,False,36
1,136951,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,2022,1,1,52,5,1,True,False,False,False,False,36
2,136952,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,2022,1,1,52,5,1,True,False,False,False,False,-1
3,136953,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,2022,1,1,52,5,1,True,False,False,False,False,32
4,136954,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,2022,1,1,52,5,1,True,False,False,False,False,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27370,164320,2022-12-31,Spain,Kagglazon,Using LLMs to Improve Your Coding,2022,12,31,52,5,31,False,False,False,False,False,747
27371,164321,2022-12-31,Spain,Kagglazon,Using LLMs to Train More LLMs,2022,12,31,52,5,31,False,False,False,False,False,715
27372,164322,2022-12-31,Spain,Kagglazon,Using LLMs to Win Friends and Influence People,2022,12,31,52,5,31,False,False,False,False,False,113
27373,164323,2022-12-31,Spain,Kagglazon,Using LLMs to Win More Kaggle Competitions,2022,12,31,52,5,31,False,False,False,False,False,697
